## 

Autor: **Radosław Dąbkowski**

Numer indeksu: **325683**

Grupa: **1**
Numer: **2**

### Wstęp



In [80]:
# -*- coding: utf-8 -*-
"""
Created on Sat Dec  3 14:08:04 2022

@author: mgrzy
"""

import numpy as np
import gpxpy
import scipy
import gpxpy.gpx
import pandas as pd

def gpx2df(gpx_file_name, new_file_name = 'route_df.csv'):
    with open(gpx_file_name, 'r') as gpx_file:
        gpx = gpxpy.parse(gpx_file)
    route_info = []
    
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                route_info.append({
                    'latitude': point.latitude,
                    'longitude': point.longitude,
                    'elevation': point.elevation,
                    'time':point.time
                })
    
    route_df = pd.DataFrame(route_info)
    
    save_df(route_df, new_file_name)

def blh2xyz(fi, lam, h) -> np.array:
    '''
    Parameters
    ----------
    fi : float
        latitude [rad].
    lam : float
        longitude [rad].
    h : float
        height [m].
    Returns
    -------
    xyz : numpy array
        x, y, z [m].
    '''
    a = 6378137.0
    e2 = 0.00669438002290
    N = a/np.sqrt(1-e2*np.sin(fi)**2)
    x = (N+h)*np.cos(fi)*np.cos(lam)
    y = (N+h)*np.cos(fi)*np.sin(lam)
    z = (N*(1-e2)+h)*np.sin(fi)
    xyz = np.array([x, y, z])
    return xyz

def save_df(route_df, new_file_name= 'route_df.csv'):
    route_df.to_csv(new_file_name, index=False)
    print(f'file saved to {new_file_name}')
def read_route_df(file):
    route_df = pd.read_csv(file)
    return route_df

In [81]:
gpx2df('Afternoon_Ride.gpx')
df = read_route_df('route_df.csv')
df.head()

file saved to route_df.csv


,latitude,longitude,elevation,time
0,53.208030,23.326793,136.4,2022-07-28 13:26:13+00:00
1,53.208035,23.326818,136.4,2022-07-28 13:26:14+00:00
2,53.208038,23.326861,136.4,2022-07-28 13:26:16+00:00
3,53.208047,23.326925,136.3,2022-07-28 13:26:17+00:00
4,53.208052,23.326948,136.3,2022-07-28 13:26:18+00:00


In [82]:
import folium

# add points that show the elevation

phis = df['latitude'].values
lambdas = df['longitude'].values
dists = []
for i in range(len(phis)-1):
    phi1 = phis[i]
    lam1 = lambdas[i]
    phi2 = phis[i+1]
    lam2 = lambdas[i+1]

    xyz1 = blh2xyz(np.deg2rad(phi1), np.deg2rad(lam1), 0)
    xyz2 = blh2xyz(np.deg2rad(phi2), np.deg2rad(lam2), 0)
    dist = np.linalg.norm(xyz1-xyz2)
    dists.append(dist)

# append the last distance
phi1 = phis[-1]
lam1 = lambdas[-1]
phi2 = phis[0]
lam2 = lambdas[0]

xyz1 = blh2xyz(np.deg2rad(phi1), np.deg2rad(lam1), 0)
xyz2 = blh2xyz(np.deg2rad(phi2), np.deg2rad(lam2), 0)
dist = np.linalg.norm(xyz1-xyz2)
dists.append(dist)

df['distance'] = np.cumsum(dists)

kron_model = np.genfromtxt('siatka_gugik-geoid2011-PL-KRON86-NH.txt', skip_header=1)
kron_model = kron_model[~np.isnan(kron_model[:, 2])]

evrf_model = np.genfromtxt('Model_quasi-geoidy-PL-geoid2021-PL-EVRF2007-NH.txt', skip_header=1)
evrf_model = evrf_model[~np.isnan(evrf_model[:, 2])]

def interpolate_geoid(model, phi, lam, tr: scipy.spatial.KDTree = None):
    '''
    Parameters
    ----------
    model : numpy array
        geoid model.
    phi : float
        latitude [deg].
    lam : float
        longitude [deg].
    tr : scipy.spatial.KDTree, optional
        KDTree. The default is None.
    Returns
    -------
    h : float
        geoid height [m].
    '''
    if tr is None:
        # https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.KDTree.html
        # https://en.wikipedia.org/wiki/K-d_tree
        # https://en.wikipedia.org/wiki/Nearest_neighbor_search
        tr = scipy.spatial.KDTree(model[:, :2])
    # find the 4 closest points
    d, idx = tr.query([phi, lam], k=4, workers=-1)
    # calculate weights
    w = 1/d
    # calculate heights
    h = model[idx, 2]
    # calculate weighted average height
    h1 = np.sum(h*w)/np.sum(w)
    return h1
    

# interpolate geoids
kronHeights = []
evrfHeights = []
kron_tr = scipy.spatial.KDTree(kron_model[:, :2])
evrf_tr = scipy.spatial.KDTree(evrf_model[:, :2])
for i in range(len(phis)):
    phi = phis[i]
    lam = lambdas[i]
    kronH = interpolate_geoid(kron_model, phi, lam, kron_tr)
    kronHeights.append(kronH)
    evrfH = interpolate_geoid(evrf_model, phi, lam, evrf_tr)
    evrfHeights.append(evrfH)

pts = list((zip(phis, lambdas, kronHeights)))

m = folium.Map(location=[df.latitude.mean(), df.longitude.mean()], zoom_start=12)

l = list(zip(df.latitude.values, df.longitude.values))

folium.PolyLine(l, color="red", weight=2.5, opacity=1).add_to(m)

# add poi points that show the elevation
# every 100th point
for i in range(0, len(pts), 100):
    folium.Marker(location=[pts[i][0], pts[i][1]],
                  popup=f'Kron86 height: {pts[i][2]:.3f} m, EVRF2007 height: {evrfHeights[i]:.3f} m',
                  icon=folium.Icon(color='green', icon='info-sign')).add_to(m)
    

m
